# Генерация датасета

В первой части проекта мы пришли к выводу, что для данной задачи лучше использовать дистанционный граф с параметром `d = 0.9`.\
Поэтому далее будем работать только с дистанционным графом.

In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx

## Вспомогательные функции

Генерация случайных величин, которые имеют распределение $Normal(0,σ)$ и $SkewNormal(α)$

In [2]:
import math

sigma0 = 1
alpha0 = 1

In [17]:
def generate_normal(n, sigma=sigma0):
  return np.random.normal(0, sigma, n)

# Плохо для параллельных вычислений
# def generate_skewnormal(n, alpha):
#   return skewnorm.rvs(alpha, size=n)

# https://stackoverflow.com/questions/36200913/generate-n-random-numbers-from-a-skew-normal-distribution-using-numpy
# literal adaption from:
# http://stackoverflow.com/questions/4643285/how-to-generate-random-numbers-that-follow-skew-normal-distribution-in-matlab
# original at:
# http://www.ozgrid.com/forum/showthread.php?t=108175
def rand_skew_norm(fAlpha, fLocation, fScale):
    sigma = fAlpha / np.sqrt(1.0 + fAlpha**2) 

    afRN = np.random.randn(2)
    u0 = afRN[0]
    v = afRN[1]
    u1 = sigma*u0 + np.sqrt(1.0 -sigma**2) * v 

    if u0 >= 0:
        return u1*fScale + fLocation 
    return (-u1)*fScale + fLocation 

def generate_skewnormal(N, skew=alpha0):
    return [rand_skew_norm(skew, 0, 1) for x in range(N)]

Функция, которая строит дистанционный граф

In [4]:
# параметр дистанцонного графа
D = 0.9

In [5]:
def build_distance_graph(vertices):
    v = np.asarray(vertices)
    n = v.size

    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i+1, n):
            if abs(v[i] - v[j]) <= D:
                G.add_edge(i, j)

    return G

Создадим функции, которые будут считать характеристики дистанционного графа

In [6]:
import networkx as nx
from collections import Counter
from networkx.algorithms.approximation.dominating_set import min_weighted_dominating_set
from networkx.algorithms import maximal_independent_set

In [7]:
def chromatic_number(G: nx.Graph) -> int:
    """
    Функция для поиска хроматического числа графа G
    """
    coloring = nx.coloring.greedy_color(G, strategy='largest_first')
    return max(coloring.values(), default=-1) + 1

In [8]:
def clique_number(G: nx.Graph) -> int:
    """
    Функция для поиска кликового числа графа G
    """
    Gc = nx.complement(G)
    coloring = nx.coloring.greedy_color(Gc, strategy='largest_first')
    counts = Counter(coloring.values())
    return max(counts.values(), default=0)

In [9]:
def max_independent_set_size(G: nx.Graph) -> int:
    """
    Функция для поиска размера максимального 
    независимого множества графа G
    """
    mis = maximal_independent_set(G)
    return len(mis)

In [10]:
def domination_number(G: nx.Graph) -> int:
    """
    Функция для поиска числа 
    доминирования для графа G
    """
    D = min_weighted_dominating_set(G)
    return len(D)

In [11]:
def min_clique_cover_size(G: nx.Graph) -> int:
    """
    Функция для поиска размера максимального
    кликового покрытия графа G
    """
    Gc = nx.complement(G)
    coloring = nx.coloring.greedy_color(Gc, strategy='largest_first')
    return len(set(coloring.values()))

## Генерация датасета

In [12]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
import networkx as nx

In [ ]:
SEED = 99
SAMPLES = 10000
N = [25, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [14]:
def generate_row(idx: int, seed: int) -> dict:
    rng = np.random.RandomState(seed + idx)
    distributions = {
        0: generate_normal,
        1: generate_skewnormal
    }

    n = rng.choice(N)
    dist_label = int(rng.choice([0, 1]))
    gen_func = distributions[dist_label]
    
    vertices = gen_func(n)
    G = build_distance_graph(vertices)
    
    return {
        'n': n,
        'distribution': dist_label,
        'chromatic_number': chromatic_number(G),
        'clique_number': clique_number(G),
        'max_independent_set_size': max_independent_set_size(G),
        'domination_number': domination_number(G),
        'min_clique_cover_size': min_clique_cover_size(G)
    }

In [20]:
def generate_dataset(num_samples=SAMPLES, seed=SEED):
    records = Parallel(n_jobs=-1, verbose=0)(
        delayed(generate_row)(i, seed)
        for i in tqdm(range(num_samples), desc="Generating rows")
    )

    df = pd.DataFrame.from_records(records, columns=[
        'n', 'distribution',
        'chromatic_number', 'clique_number',
        'max_independent_set_size',
        'domination_number', 'min_clique_cover_size'
    ])
    df.to_csv('data/distance_graph_dataset_2.csv', index=False)